# Тестовое задание для KazanExpress

## Цель исследования

На маркетплейсе ежедневно появляются сотни новых товаров. Проверить правильность заполнения информации обо всех товарах сразу невозможно. Неверно определенная категория зачастую приводит к потенциально упущенной прибыли как со стороны продавца, так и со стороны маркетплейса. 

Необходимо разработать модель, которая будет предсказывать правильную категорию товара на основе имеющихся данных (текст + картинки)

Чтобы перейти к воспроизведению результата, минуя обучение финальной и промежуточной моделей, нужно выполнить ячейки из этапов 
- "Импорт библиотек", 
- "Загрузка и осмотр данных", 
- "План работы и предобработка данных"

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import json
import time
import pickle

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
import pymorphy2


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felixalex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка и осмотр данных

Загрузка данных

In [2]:
parquet_file_test = r'test.parquet'
parquet_file_cat = r'train_category.parquet'

In [3]:
train_data = pd.read_parquet(parquet_file_cat, engine = 'auto')
test_data = pd.read_parquet(parquet_file_test, engine = 'auto')

Осмотр данных

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91120 entries, 0 to 99992
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     91120 non-null  int64  
 1   category_id    91120 non-null  int64  
 2   sale           91120 non-null  bool   
 3   shop_id        91120 non-null  int64  
 4   shop_title     91120 non-null  object 
 5   rating         91120 non-null  float64
 6   text_fields    91120 non-null  object 
 7   category_name  91120 non-null  object 
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 5.6+ MB


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16860 entries, 1 to 24995
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_id   16860 non-null  int64  
 1   sale         16860 non-null  bool   
 2   shop_id      16860 non-null  int64  
 3   shop_title   16860 non-null  object 
 4   rating       16860 non-null  float64
 5   text_fields  16860 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 806.8+ KB


Все значения заполнены, пропусков в данных нет, объем тренировочной выборки - 91120 объектов, тестовой - 16860 объектов.

In [6]:
print('Уникальных категорий:',len(train_data['category_id'].unique()))

Уникальных категорий: 874


In [7]:
train_data['category_id'].value_counts()

11937    6590
14922    3709
13651    1463
13143    1460
12980    1222
         ... 
12808       2
12901       1
11549       1
11875       1
12836       1
Name: category_id, Length: 874, dtype: int64

В целевом признаке присутствует сильный дисбаланс классов

In [8]:
train_data.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
4,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
5,1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


Имеем следующие признаки
 - `product_id` - id товара
 - `category_id` - id категории, целевой признак, который необходимо предсказать в тестовой выборке
 - `sale` - признак, показывающий, проходит ли распродажа по данному товару
 - `shop_id` - id магазина
 - `shop_title` - название магазина
 - `rating` - рейтинг товара
 - `text_fields` - описание товара

Признаков не так много, для их отбора можем использовать логику.
- `product_id` - просто индекс, который никак не влияет на целевой признак - можем убрать или использовать как индекс
- `sale` - в один день в распродажу могут попасть одни товары, в другой день - другие. Даже если именно сегодня признак позволит повысить целевую метрику, то совсем не обязательно, что через пару месяцев он поможет верно предсказать категорию (например, сегодня распродажа зимней одежды, а через полгода - велосипедов, а распродажа будет увеличивать вероятность предсказания первой категории) - не нужен
- `shop_id` и `shop_title` - по сути, дублирующие друг друга признаки. Оставим shop_title, т.к. в названии магазина может присутствовать подсказка для категории
- `rating` - оценка пользователей, которая, возможно, и имеет зависимость с категорией товара, но отсутсвует от слова совсем у нового товара.
- `text_fields` - Словарь с описанием, включающий в себя:
    - `title` - название товара
    - `description` - описание товара
    - `attributes`, `custom_characteristics`, `defined_characteristics`, `filters` - иные свойства
    
По этому полю как раз и будем определять категорию.

In [9]:
train_data = train_data.set_index('product_id')

## План работы и предобработка данных

Необходимо:
1. Извлечь признаки, по которым будем определять категорию товара
2. Провести первичную предобработку (очистку, лемматизацию)

Из текста извлечем признаки из названия товара (`title`), остальное отбросим, т.к. в описании запросто может присутствовать что-то вроде: "футболка отлично подходит к штанам" или "в нашем магазине Вы найдете все для оформления праздника: свечи, феерверки, мишуру" и т.д., что только помешает верно предсказать категорию.

Т.к. `text_fields` представляет из себя словарь, то просто извлечем значение по ключу

In [10]:
def title_extraction(string):
    title = json.loads(string)['title']
    return title

Далее нам необходимо очистить текст от ненужных символов. И хотя `TfidfVectorizer`, который мы будем использовать далее, прекрасно делает это сам, лишние знаки препинания помешают процессу лемматизации. Поэтому следующую функцию применим перед лемматизацией. Английские символы удалять не будем, т.к. они тоже могут иметь смысл при определении категории

In [11]:
def clear_text(string):
    string_wo_letters = re.sub(r'[^а-яА-ЯёЁa-zA-Z]',' ',string)
    only_letters_list = string_wo_letters.split()
    result = ' '.join(only_letters_list)
    return result

Необходимо провести лемматизацию (процесс приведения слова к исходной форме). Это позволит сократить количество признаков (т.к. однокоренные слова будут приведены к одной форме). Будем использовать библиотеку `pymorphy2`.

In [12]:
morph = pymorphy2.MorphAnalyzer()

In [13]:
def lemmatize(text):
    words = text.split()
    lemm_text = ''
    for word in words:
        lemm_text += ' ' + morph.parse(word)[0].normal_form
    return lemm_text

Создадим колонку `title` и проведем необходимые преобразования

In [14]:
train_data['title'] = train_data['text_fields'].apply(title_extraction)
train_data['title'] = train_data['title'].apply(clear_text)
train_data['title'] = train_data['title'].apply(lemmatize)

Добавим название магазина к признакам (лемматизировать его не будем, пусть он представляет из себя отдельный признак) и посмотрим на получившийся датасет

In [15]:
train_data['title'] = train_data['title'] + ' ' + train_data['shop_title']
train_data.head()

,category_id,sale,shop_id,shop_title,rating,text_fields,category_name,title
product_id,,,,,,,,
325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...,зарядный кабель borofone bx lightning для айф...
888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...,трус sela Sela
1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...,гуашь юный волшебник цвет по мл больший баноч...
1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...,колба для кальян крафт разный цвет вася-nicotine
1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...,пижама женский однотонный с шорты Lim Market


Было:

In [16]:
train_data['text_fields'].iloc[5]

'{"title": "Накладка-чехол на стики для джойстика", "description": "<p><span style=\\"color: rgb(85, 85, 85);\\">Накладки на стики геймпада - особо востребованный геймерами аксессуар благодаря своему предназначению и низкой стоимости. Накладки выполнены из качественного, как правило, силикона или мягкой резины, их нереально стереть во время игры, а одеваются накладки на стики с натяжением, что исключает их спадание во время игры или транспортировки контроллера/геймпада.</span></p><p><br></p><p>Подойдут для следующих консолей:</p><p>Контроллер Dualshock 4 PS4 Pro</p><p>Контроллер Dualshock 4 PS4 тонкий контроллер</p><p>Контроллер Dualshock 4 PS4</p><p>Контроллер Dualshock 3 PS3 проводной/беспроводной</p><p>Контроллер Dualshock 2 PS2 проводной/беспроводной</p><p>Контроллер Xbox One X Контроллер</p><p>Контроллер Xbox One S/тонкий</p><p>Контроллер Xbox One Elite</p><p>Контроллер Xbox One</p><p>Проводной/беспроводной контроллер для Xbox 360</p><p>Контроллер NS switch Pro</p><p><br></p><p><i

Стало

In [17]:
train_data['title'].iloc[5]

' накладка чехол на стик для джойстик Device Advice'

Делаем то же самое с тестовыми данными

In [18]:
test_data['title'] = test_data['text_fields'].apply(title_extraction)
test_data['title'] = test_data['title'].apply(clear_text)
test_data['title'] = test_data['title'].apply(lemmatize)
test_data['title'] = test_data['title'] + ' ' + test_data['shop_title']
test_data.head()

,product_id,sale,shop_id,shop_title,rating,text_fields,title
1,1997646,False,22758,Sky_Electronics,5.000000,"{""title"": ""Светодиодная лента Smart led Strip ...",светодиодный лента smart led strip light с пу...
2,927375,False,17729,Di-Di Market,4.405941,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor...",стекло плёнка керамик матовый honor lite pro ...
3,1921513,False,54327,VisionStore,4.000000,"{""title"": ""Проводные наушники с микрофоном jac...",проводной наушник с микрофон jack ios android...
4,1668662,False,15000,FORNAILS,5.000000,"{""title"": ""Декоративная табличка \""Правила кух...",декоративный табличка правило кухня подставка...
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"{""title"": ""Подставка под ложку керамическая, п...",подставка под ложка керамический подложка клу...


Извлечем целевой признак и фичи

In [19]:
features = train_data['title']
target = train_data['category_id']

Пропустить все обучение моделей и перейти к воспроизведению результата [можно тут](#1)

## Обучение модели

Для того, чтобы мы могли обучить модель, необходимо перевести данные в числовой формат, будем пользоваться TF-IDF и `TfidfVectorizer`.
Сама формула меры имеет следующий вид:
$$
TFIDF = TF \cdot IDF
$$

Множитель `TF` отвечает за количество упоминаний слова в отдельном тексте
$$
TF = {t \over n}
$$
где:
- t — количество употребления слова, 
- а n — общее число слов в тексте.

Множитель `IDF` отражает частоту употребления слова во всём корпусе
$$
IDF = log_{10}({D \over d})
$$

где:
- D - число текстов в корпусе 
- d - количества текстов, в которых встречается искомое слово

Таким образом, бо'льшая величина TF-IDF говорит о б'ольшей уникальности слова во всем корпусе текстов

Установим список стоп-слов для `TfidfVectorizer`, так мы исключим местоимения, предлоги и т.д.

In [20]:
stopwords = set(nltk_stopwords.words('russian'))

`TfidfVectorizer` должен обучаться <u>только</u> на тренировочном датасете. То есть, имеем 2 варианта:

- a. Используем пайплайн и кроссвалидацию
    - в этом случае имеем проблему с апсемплингом. Несколько категорий имеют всего по одному объекту, а метод SMOTE работает по принципу ближайших соседей (по дефолту пять). То есть нам будет необходимо <u>отбросить</u> довольно много категорий перед обучением пайплайна. Кроссвалидация даст заведомо завышенный результат. 
    
- b. Разбиваем датасет на тренировочную и валидационную выборки, обучаем модель на тренировочной, предсказываем на валидационной
    - имеем проблему с подбором гиперпараметров, а именно невозможностью оценить стандартное отклонение при разбиении на фолды.

Датасет довольно объемный и подбор гиперпараметров может занять очень длительное время. Поэтому объединим 2 решения:
- 1) Выделяем из датасета 20% объектов
- 2) Убираем малопредставленные категории
- 3) Строим пайплайн (TF-IDF, SMOTE, Модель)
- 4) Подбираем гиперпараметры с помощью GridSearchCV. Сознательно идем на эту ошибку(кроссвалидация на сэмплированных данных), т.к. знаем, что исправим ее в следующих шагах
- 5) Делим датасет в пропорциях (85%-15%)
- 6) Обучаем модель с гиперпараметрами, подобранными на шаге 4
- 7) Тестируем модель на валидационной выборке

Разбиваем данные на обучающую и тестовую выборки

In [21]:
x_train, x_valid, y_train, y_valid = train_test_split(features, target, train_size=0.1, test_size=0.02, random_state=12345)
print(x_train.shape)
print(y_train.shape)

(9112,)
(9112,)


Уберем малопредставленные категории, для этого создадим таблицу counts, которую используем как фильтр. Пусть SMOTE делает апсемплинг по 3 ближайшим соседям, а кроссваладиция проходит по 3 фолдам, тогда нам нужно убрать категории менее чем с 10 объектами

In [22]:
counts = y_train.value_counts()
y_train = y_train[y_train.isin(counts[counts>10].index)]
x_train = x_train[y_train.index]
print(x_train.shape)
print(y_train.shape)

(7022,)
(7022,)


Убрано более 20% объектов. Печально, но если мы хотим использовать апсемплинг при подборе гиперпараметров с помощью кроссвалидации по 10 часов, то этого не избежать.
Строим пайплайн. Сэмплеру зададим увеличение выборки минорных классов до 100 объектов

In [23]:
pipeline = Pipeline(
    [
        ('vectorizer', TfidfVectorizer(stop_words = stopwords)),
        ('sampler', SMOTE(sampling_strategy={i: 100 for i in y_train.value_counts().tail(200).index}, 
                          k_neighbors=3, 
                          random_state=12345)),
        ('classifier', DummyClassifier())
    ]
)
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(stop_words={'а', 'без', 'более', 'больше',
                                             'будет', 'будто', 'бы', 'был',
                                             'была', 'были', 'было', 'быть',
                                             'в', 'вам', 'вас', 'вдруг', 'ведь',
                                             'во', 'вот', 'впрочем', 'все',
                                             'всегда', 'всего', 'всех', 'всю',
                                             'вы', 'где', 'да', 'даже', 'два', ...})),
                ('sampler',
                 SMOTE(k_neighbors=3, random_state=12345,
                       sampling_strategy={2599: 100, 2601: 100, 2728: 100,
                                          2730: 100, 2733: 100, 2740: 100,
                                          2741: 100, 2742: 100, 2743: 100,
                                          2744: 100, 2747: 100, 2748: 100,
                                          2752: 100, 2753: 100, 2754: 100,
                                          2755: 100, 2757: 100, 2761: 100,
                                          2763: 100, 2765: 100, 2767: 100,
                                          2769: 100, 2771: 100, 2775: 100,
                                          2777: 100, 2778: 100, 2791: 100,
                                          2802: 100, 2803: 100, 2804: 100, ...})),
                ('classifier', DummyClassifier())])

Задаем сетку гиперпараметров и обучаем модель. Из моделей проверим метод опорных векторов и логистическую регрессию с разной регуляризацией. Также посмотрим как на результат влияют биграммы.

In [24]:
params = [
    {
        'classifier': [SVC(random_state=12345, kernel='linear')],
        'classifier__C': [0.1, 1, 10, 50, 100],
        'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
    },
    {
        'classifier': [LogisticRegression(max_iter=1000, random_state=12345)],
        'classifier__C': [0.1, 1, 10, 50, 100],
        'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)]
    },
    {
        'classifier': [DummyClassifier(random_state=12345)]
    }   
]

In [25]:
grid_search_svc_lr = GridSearchCV(estimator=pipeline, 
                               param_grid=params,
                               cv=3,
                               scoring='f1_weighted',
                               n_jobs=4,
                               verbose=3)

In [26]:
grid_search_svc_lr.fit(x_train,y_train)
grid_search_svc_lr.best_score_

Fitting 3 folds for each of 31 candidates, totalling 93 fits


0.832977246072352

Проанализируем зависимость метрики от гиперпараметров

In [27]:
pd.concat(
    [pd.DataFrame(grid_search_svc_lr.cv_results_['params']),
     pd.Series(grid_search_svc_lr.cv_results_['mean_test_score'],name='score'),
     pd.Series(grid_search_svc_lr.cv_results_['std_test_score'],name='std')],
    axis=1
).sort_values('score', ascending=False)

,classifier,classifier__C,vectorizer__ngram_range,score,std
21,"LogisticRegression(C=10, max_iter=1000, random...",10.0,"(1, 1)",0.832977,0.003670
27,"LogisticRegression(C=10, max_iter=1000, random...",100.0,"(1, 1)",0.830011,0.003700
24,"LogisticRegression(C=10, max_iter=1000, random...",50.0,"(1, 1)",0.829869,0.003992
18,"LogisticRegression(C=10, max_iter=1000, random...",1.0,"(1, 1)",0.825525,0.001905
28,"LogisticRegression(C=10, max_iter=1000, random...",100.0,"(1, 2)",0.821442,0.003981
25,"LogisticRegression(C=10, max_iter=1000, random...",50.0,"(1, 2)",0.821207,0.004631
3,"SVC(kernel='linear', random_state=12345)",1.0,"(1, 1)",0.821186,0.009454
9,"SVC(kernel='linear', random_state=12345)",50.0,"(1, 1)",0.820856,0.007233
6,"SVC(kernel='linear', random_state=12345)",10.0,"(1, 1)",0.820789,0.008608
12,"SVC(kernel='linear', random_state=12345)",100.0,"(1, 1)",0.820623,0.008057


Биграммы не дают прироста, а только ухудшают качество. В топе логистическая регрессия с C=10, и SVC (сила решуляризации почти не влияет), отклонение от ожидаемого результата невелико: в основном не более 1%

Делим датасет в соотношении 85%-15% и обучаем финалистов на всем датасете и смотрим результаты

In [28]:
x_train, x_valid, y_train, y_valid = train_test_split(features, target, test_size = 0.15, random_state=12345)
print(x_train.shape)
print(y_train.shape)

(77452,)
(77452,)


Оставим для SMOTE 3 ближайших соседа

In [29]:
counts = y_train.value_counts()
y_train = y_train[y_train.isin(counts[counts>3].index)]
x_train = x_train[y_train.index]
print(x_train.shape)
print(y_train.shape)

(77377,)
(77377,)


Отбросили всего 75 объектов из обучающей выборки.

Обучаем vectorizer на тренировочной выборке, применяем transform на тренировочной и валидационной выборке

In [30]:
vectorizer = TfidfVectorizer(stop_words = stopwords, ngram_range=(1, 1))
x_train_vectorized = vectorizer.fit_transform(x_train)
x_valid_vectorized = vectorizer.transform(x_valid)

Семплируем тренировочную выборку, увеличим минорный класс до 300 категорий

In [31]:
y_train.value_counts().tail(805)

12374    317
12914    308
13954    306
13429    301
2755     297
        ... 
12776      4
11977      4
14585      4
15040      4
12011      4
Name: category_id, Length: 805, dtype: int64

In [32]:
sampler = SMOTE(sampling_strategy={i: 300 for i in y_train.value_counts().tail(801).index}, 
                k_neighbors=3, 
                random_state=12345)
x_train_vectorized, y_train = sampler.fit_resample(x_train_vectorized, y_train)

Обучаем модель логистической регрессии

In [33]:
%%time
clf_lr = LogisticRegression(max_iter=1000, random_state=12345, C=10)
clf_lr.fit(x_train_vectorized, y_train)

CPU times: total: 46min 26s
Wall time: 40min 6s


LogisticRegression(C=10, max_iter=1000, random_state=12345)

In [34]:
%%time
y_preds_lr = clf_lr.predict(x_valid_vectorized)
tf_idf_score_f1_lr = f1_score(y_valid, y_preds_lr, average='weighted')
print(f'Результат логистической регрессии: {tf_idf_score_f1_lr}')

Результат логистической регрессии: 0.8575165563542101
CPU times: total: 234 ms
Wall time: 242 ms


Обучаем модель SVC

In [35]:
%%time
clf_svc = SVC(C=100, kernel='linear', random_state=12345)
clf_svc.fit(x_train_vectorized, y_train)

CPU times: total: 18min 55s
Wall time: 19min 6s


SVC(C=100, kernel='linear', random_state=12345)

In [36]:
%%time
y_preds_svc = clf_svc.predict(x_valid_vectorized)
tf_idf_score_f1_svc = f1_score(y_valid, y_preds_svc, average='weighted')
print(f'Результат метода опорных векторов: {tf_idf_score_f1_svc}')

Результат метода опорных векторов: 0.8468113226273071
CPU times: total: 37min 33s
Wall time: 37min 48s


**Вывод:**
- Модель логистической регрессии показывает результат немного лучше, чем метод опорных векторов. 
- Модель логистической регрессии имеет практически мгновенное время предсказывания. 


Поэтому выберем ее в качестве финальной. Посмотрим на качество предсказаний. 

## Анализ результатов работы модели

Составим таблицу - фильтр с неверными ответами

In [37]:
false_answers_filter = pd.Series(y_valid != y_preds_lr)

С помощью этой таблицы составим датафрейм, тут все предсказанные категории - неверные

добавляем поля:
- `true_cat_id` - поле с id верной категории
- `text` - поле с изначальным описанием товара
- `true_cat_name` - поле с названием верной категории
- `category_name` - поле с названием предсказанной категории

In [38]:
false_preds_valid = pd.DataFrame(y_preds_lr,index=false_answers_filter.index,columns=['predicted_cat_id'])
false_preds_valid = false_preds_valid[false_answers_filter]
false_preds_valid[['true_cat_id','text','true_cat_name']] = train_data[['category_id','text_fields','category_name']]
false_preds_valid = false_preds_valid.join(train_data.groupby('category_id').min()['category_name'], on='predicted_cat_id')
false_preds_valid

,predicted_cat_id,true_cat_id,text,true_cat_name,category_name
product_id,,,,,
1823163,13674,12667,"{""title"": ""Автоматические контейнеры для круп""...",Все категории->Товары для дома->Товары для кух...,Все категории->Товары для дома->Товары для кух...
741807,13651,2769,"{""title"": ""Основа для броши. Английская булавк...",Все категории->Хобби и творчество->Рукоделие->...,Все категории->Хобби и творчество->Рукоделие->...
378151,2880,13651,"{""title"": ""Бусины 6 мм из полимерной глины, ро...",Все категории->Хобби и творчество->Рукоделие->...,Все категории->Хобби и творчество->Лепка->Глин...
920620,2744,14873,"{""title"": ""Держатель для зубных щеток и пасты ...",Все категории->Товары для дома->Хозяйственные ...,Все категории->Товары для дома->Хозяйственные ...
1170375,12282,11956,"{""title"": ""Футболки с аниме принтами"", ""descri...",Все категории->Одежда->Женская одежда->Футболк...,Все категории->Одежда->Мужская одежда->Футболк...
...,...,...,...,...,...
819054,13033,12640,"{""title"": ""Картина по номерам на холсте с подр...",Все категории->Хобби и творчество->Создание ка...,Все категории->Хобби и творчество->Создание ка...
1838575,13171,2838,"{""title"": ""Брюки-джоггеры джинсовые, мужские, ...",Все категории->Одежда->Мужская одежда->Спортив...,Все категории->Одежда->Мужская одежда->Брюки и...
902541,14869,2747,"{""title"": ""Набор из 5 бирок-шильдиков к нового...",Все категории->Товары для дома->Товары для пра...,Все категории->Товары для дома->Товары для пра...


Посмотрим на 10 неправильных предсказаний

In [39]:
for i in range(10):
    print(i)
    print(f"Правильная категория: {false_preds_valid.iloc[i]['true_cat_name']} \
            \nПредсказанная категория: {false_preds_valid.iloc[i]['category_name']} \
            \n\nТекст: {false_preds_valid.iloc[i]['text']}, \n")

0
Правильная категория: Все категории->Товары для дома->Товары для кухни->Хранение продуктов->Банки и крышки             
Предсказанная категория: Все категории->Товары для дома->Товары для кухни->Хранение продуктов->Контейнеры и ланч-боксы             

Текст: {"title": "Автоматические контейнеры для круп", "description": "<p>Контейнеры для сыпучих продуктов</p>", "attributes": ["Этот органайзер для круп можно использовать, как контейнер для хранения круп, сыпучих продуктов, сухих завтраков, специй, орехов, разных круп.", "Общий объем - 10 л.", "Диспенсер для сыпучих продуктов с шестью отсеками."], "custom_characteristics": {}, "defined_characteristics": {}, "filters": {}}, 

1
Правильная категория: Все категории->Хобби и творчество->Рукоделие->Создание украшений             
Предсказанная категория: Все категории->Хобби и творчество->Рукоделие->Материалы для рукоделия             

Текст: {"title": "Основа для броши. Английская булавка с петлями. Булавка", "description": "<p>Булавка 

Ошибки, в основном, представлены категориями, где весь контекст заключен в описании товара, а не в его названии. 

Но есть и близкие по смыслу категории типа `Рукоделие->Шитье->Инструменты` и `Рукоделие->Инструменты для рукоделия` довольно близки по смыслу

Составим сводную таблицу, которая покажет какие категории путает модель, отсортируем по числу ошибок.

In [40]:
false_preds_valid.pivot_table(index=['category_name','true_cat_name'], 
                              values='predicted_cat_id', 
                              aggfunc='count').sort_values('predicted_cat_id', ascending=False).head(10)

,,predicted_cat_id
category_name,true_cat_name,
"Все категории->Одежда->Женская одежда->Колготки, носки, чулки->Носки и подследники",Все категории->Одежда->Мужская одежда->Носки->Носки и подследники,31
Все категории->Одежда->Мужская одежда->Носки->Носки и подследники,"Все категории->Одежда->Женская одежда->Колготки, носки, чулки->Носки и подследники",24
Все категории->Одежда->Женская одежда->Футболки и топы->Футболки,Все категории->Одежда->Мужская одежда->Футболки и майки->Футболки,15
Все категории->Хобби и творчество->Рукоделие->Материалы для рукоделия,Все категории->Хобби и творчество->Рукоделие->Создание украшений,15
Все категории->Электроника->Смартфоны и телефоны->Аксессуары и запчасти->Наклейки для телефонов,Все категории->Хобби и творчество->Стикеры,12
Все категории->Хобби и творчество->Рукоделие->Создание украшений,Все категории->Хобби и творчество->Рукоделие->Материалы для рукоделия,11
Все категории->Электроника->Компьютерная техника->Комплектующие для компьютерной техники->Прочая электротехника,Все категории->Электроника->Аксессуары для электроники->Преобразователи напряжения,10
Все категории->Хобби и творчество->Стикеры,Все категории->Товары для дома->Декор и интерьер->Оформление интерьера->Наклейки и пленки,10
Все категории->Одежда->Женская одежда->Платья->Платья повседневные,Все категории->Одежда->Женская одежда->Платья->Платья вечерние,9


Первые 3 места занимают ошибки, где модель спутала мужскую и женскую одежду.

## Дообучение финальной модели

Сделаем refit финальной модели на всем датасете и предскажем значения тестовой выборки

In [41]:
x_train = features.copy()
y_train = target.copy()
print(x_train.shape)
print(y_train.shape)

(91120,)
(91120,)


Переодучим семплер и векторайзер

In [42]:
counts = y_train.value_counts()
y_train = y_train[y_train.isin(counts[counts>3].index)]
x_train = x_train[y_train.index]
print(x_train.shape)
print(y_train.shape)

(91080,)
(91080,)


In [43]:
vectorizer = TfidfVectorizer(stop_words = stopwords, ngram_range=(1, 1))
x_train_vectorized = vectorizer.fit_transform(x_train)
x_train_vectorized.shape

(91080, 27901)

Сделаем апсемплинг минорного класса до минимального значения в 300 объектов

In [44]:
y_train.value_counts().tail(803)

12755    304
11790    303
12683    302
14541    298
11756    297
        ... 
13241      4
11642      4
12011      4
11749      4
11594      4
Name: category_id, Length: 803, dtype: int64

In [45]:
sampler = SMOTE(sampling_strategy={i: 300 for i in y_train.value_counts().tail(800).index}, 
                k_neighbors=3, 
                random_state=12345)
x_train_vectorized, y_train = sampler.fit_resample(x_train_vectorized, y_train)

Обучаем модель

In [46]:
%%time
clf_lr = LogisticRegression(max_iter=1000, random_state=12345, C=10)
clf_lr.fit(x_train_vectorized, y_train)

CPU times: total: 50min 1s
Wall time: 43min 37s


LogisticRegression(C=10, max_iter=1000, random_state=12345)

Сохраняем векторайзер и модель в файл 
<a id="1"></a>

In [47]:
model_name = 'final_model.sav'
pickle.dump(clf_lr, open(model_name, 'wb'))
vectorizer_name = 'vectorizer.sav'
pickle.dump(vectorizer, open(vectorizer_name, 'wb'))

In [48]:
# model_name = 'final_model.sav'
# clf_lr = pickle.load(open(model_name, 'rb'))
# vectorizer_name = 'vectorizer.sav'
# vectorizer = pickle.load(open(vectorizer_name, 'rb'))

Преобразуем тестовые данные и предсказываем категорию

In [49]:
x_test = test_data['title']
x_test_vectorized = vectorizer.transform(x_test)
y_test = clf_lr.predict(x_test_vectorized)
y_test

array([13495, 14922,  2803, ..., 13651,  2740, 11757], dtype=int64)

In [50]:
result = pd.DataFrame(test_data['product_id'])
result['predicted_category_id '] = y_test
result

,product_id,predicted_category_id
1,1997646,13495
2,927375,14922
3,1921513,2803
4,1668662,12524
5,1467778,13887
...,...,...
24987,1914264,11645
24988,1310569,12357
24989,978095,13651
24992,797547,2740


In [51]:
result.to_parquet('result.parquet')

In [1]:
!pip3 freeze > requirements.txt